In [89]:
import pandas as pd
import os
import json as jsonlib
from typing import Tuple

# Obtenemos el CSV con competiciones
cdir = os.getcwd()
utils = os.path.join(os.path.abspath(os.path.join(cdir, '..', '..')), 'utils')
comps = pd.read_csv(os.path.join(utils, 'comps.csv'), sep=';')

# JSON con temporadas deseadas
with open(os.path.join(utils, 'des_seasons.json'), 'r', encoding='utf-8') as f:
    desired_seasons = jsonlib.load(f)

# Lector de JSON
def json_to_dict(json_path: str) -> dict:
    with open(json_path, "r", encoding="utf-8") as f:
        dict = jsonlib.load(f)
    return dict

In [100]:
# Generamos el dataframe de temporadas de una liga
def get_seasons_df(league_id: int, available_seasons_json: dict) -> pd.DataFrame:

    # Info que necesitaremos
    all_seasons = available_seasons_json.get('seasons', {})

    # Obtenemos información
    if not all_seasons:
        return pd.DataFrame()
    
    # Lista para concatenar info
    rows = []

    for season in all_seasons:

        season_key = season.get('year', '').replace('/','')

        if season_key in desired_seasons:
            rows.append({'league': league_id,
                         'year': season_key,
                         'season_name': season.get('name', '')})

    return pd.DataFrame(rows)

# Función para obtener las tablas de clasificación
def obtain_standings_tables(league_id: int, season_key: str, standings_json: dict) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    # Función para obtener la tabla con standings a partir de casa, away, etc
    def partial_standing(st_table: dict) -> pd.DataFrame:
        list_info = []
        for team in st_table:
            list_info.append({'position': team.get('position', 0),
                            'team': team.get('team', {}).get('name', ''),
                            'team_slug': team.get('team', {}).get('slug', ''),
                            'promotion': team.get('promotion', {}).get('text', ''),
                            'points': team.get('points', 0),
                            'matches': team.get('matches', 0),
                            'wins': team.get('wins', 0),
                            'losses': team.get('losses', 0),
                            'draws': team.get('draws', 0),
                            'scores_for': team.get('scoresFor', 0),
                            'scores_against': team.get('scoresAgainst', 0)})
        return pd.DataFrame(list_info)

    # Diferenciamos las tres tablas
    total_st = standings_json.get('total', {}).get('standings', [])[0].get('rows', [])
    home_st = standings_json.get('home', {}).get('standings', [])[0].get('rows', [])
    away_st = standings_json.get('away', {}).get('standings', [])[0].get('rows', [])

    # Obtenemos tablas
    total_st_df = partial_standing(st_table=total_st)
    home_st_df = partial_standing(st_table=home_st)
    away_st_df = partial_standing(st_table=away_st)

    # Añadimos liga y season
    for df in [total_st_df, home_st_df, away_st_df]:
        df.insert(0, 'league', league_id)
        df.insert(1, 'season', season_key)
    
    return total_st_df, home_st_df, away_st_df

# Procesamos el JSON de jugadores y la carpeta con más información
def players_processing(player_json: dict, players_dict_path: str) -> pd.DataFrame:

    players_list = []

    # Para cada jugador
    for player in player_json.get('players', []):

        # ID
        player_id = player.get('playerId')
        if player_id:
            player_dict = {'id': player_id, 'name': player.get('playerName', '')}

            # Si existe el json
            player_json_path = os.path.join(players_dict_path, f'{player_id}.json')
            if os.path.exists(player_json_path):
                player_json = json_to_dict(player_json_path)

                # Positions
                positions = player_json.get('player', {}).get('positionsDetailed', [])

                # Añadimos valores al diccionario
                player_dict['slug'] = player_json.get('player', {}).get('slug', '')
                player_dict['short_name'] = player_json.get('player', {}).get('shortName', '')
                player_dict['team'] = player_json.get('player', {}).get('team', {}).get('name', '')
                player_dict['country'] = player_json.get('player', {}).get('country', {}).get('name', '')
                player_dict['position'] = positions[0]
                player_dict['second_position'] = positions[1] if len(positions) > 1 else ''
                player_dict['third_position'] = positions[2] if len(positions) > 2 else ''
                player_dict['weight'] = player_json.get('player', {}).get('weight', 0)
                player_dict['height'] = player_json.get('player', {}).get('height', 0)
                player_dict['shirt_number'] = player_json.get('player', {}).get('shirtNumber', 0)
                player_dict['pref_foot'] = player_json.get('player', {}).get('preferredFoot', '')
                player_dict['date_birth'] = player_json.get('player', {}).get('dateOfBirthTimestamp', 0)
                player_dict['contract_until'] = player_json.get('player', {}).get('contractUntilTimestamp', 0)
                player_dict['market_value'] = player_json.get('player', {}).get('proposedMarketValue', 0)

            players_list.append(player_dict)
    
    return pd.DataFrame(players_list)

# Procesamos el JSON de teams y la carpeta con más información
def teams_processing(team_json: dict, teams_dict_path: str) -> pd.DataFrame:

    teams_list = []

    # Para cada equipo
    for team in team_json.get('teams', []):

        # ID
        team_id = team.get('id')
        if team_id:
            team_dict = {'id': team_id, 'name': team.get('name', '')}

            # Si existe el json
            team_json_path = os.path.join(teams_dict_path, f'{team_id}.json')
            if os.path.exists(team_json_path):
                team_json = json_to_dict(team_json_path)

                # Añadimos información al diccionario
                team_dict['slug'] = team_json.get('team', {}).get('slug', '')
                team_dict['short_name'] = team_json.get('team', {}).get('shortName', '')
                team_dict['full_name'] = team_json.get('team', {}).get('fullName', '')
                team_dict['code'] = team_json.get('team', {}).get('nameCode', '')
                team_dict['manager'] = team_json.get('team', {}).get('manager', {}).get('name', '')
                team_dict['venue'] = team_json.get('team', {}).get('venue', {}).get('name', '')
                team_dict['country'] = team_json.get('team', {}).get('country', {}).get('name', '')
                team_dict['primary_colour'] = team_json.get('team', {}).get('teamColors', {}).get('primary', '')
                team_dict['secondary_colour'] = team_json.get('team', {}).get('teamColors', {}).get('secondary', '')
                team_dict['text_colour'] = team_json.get('team', {}).get('teamColors', {}).get('text', '')
                team_dict['foundation'] = team_json.get('team', {}).get('foundationDateTimestamp', 0)

            teams_list.append(team_dict)
    
    return pd.DataFrame(teams_list)

# Procesamos el JSON de teams y la carpeta con más información
def venues_processing(venue_json: dict, venues_dict_path: str) -> pd.DataFrame:

    venues_list = []

    # Para cada equipo
    for venue in venue_json.get('venues', []):

        # ID
        venue_id = venue.get('id')
        if venue_id:
            venue_dict = {'id': venue_id, 'name': venue.get('name', '')}

            # Si existe el json
            venue_json_path = os.path.join(venues_dict_path, f'{venue_id}.json')
            if os.path.exists(venue_json_path):
                venue_json = json_to_dict(venue_json_path)

                # Añadimos información al diccionario
                venue_dict['slug'] = venue_json.get('venue', {}).get('slug', '')
                venue_dict['capacity'] = venue_json.get('venue', {}).get('capacity', 0)
                venue_dict['city'] = venue_json.get('venue', {}).get('city', {}).get('name', '')
                venue_dict['country'] = venue_json.get('venue', {}).get('country', {}).get('name', '')
                venue_dict['latitude'] = venue_json.get('venue', {}).get('venueCoordinates', {}).get('latitude', 0.0)
                venue_dict['longitude'] = venue_json.get('venue', {}).get('venueCoordinates', {}).get('longitude', 0.0)
                venue_dict['matches'] = venue_json.get('statistics', {}).get('matches', 0)
                venue_dict['home_goals'] = venue_json.get('statistics', {}).get('homeTeamGoalsScored', 0)
                venue_dict['away_goals'] = venue_json.get('statistics', {}).get('awayTeamGoalsScored', 0)
                venue_dict['avg_red_cards_game'] = venue_json.get('statistics', {}).get('avgRedCardsPerGame', 0)
                venue_dict['avg_ck_game'] = venue_json.get('statistics', {}).get('avgCornerKicksPerGame', 0)
                venue_dict['home_wins_perc'] = venue_json.get('statistics', {}).get('homeTeamWinsPercentage', 0)
                venue_dict['away_wins_perc'] = venue_json.get('statistics', {}).get('awayTeamWinsPercentage', 0)
                venue_dict['draws_perc'] = venue_json.get('statistics', {}).get('drawsPercentage', 0)
                
            venues_list.append(venue_dict)
    
    return pd.DataFrame(venues_list)

# Dataframe con la información de los entrenadores
def managers_processing(managers_dict_path: str) -> pd.DataFrame:

    list_managers = []

    # Para cada manager que tenemos
    for manager_path in os.listdir(managers_dict_path):
        
        # Full path y json
        full_path = os.path.join(managers_dict_path, manager_path)
        manager_json = json_to_dict(json_path=full_path).get('manager')

        if manager_json:
            list_managers.append({'id': manager_json.get('id', 0),
                                  'name': manager_json.get('name', ''),
                                  'slug': manager_json.get('slug', ''),
                                  'short_name': manager_json.get('shortName', ''),
                                  'team': manager_json.get('team', {}).get('name', ''),
                                  'pref_formation': manager_json.get('preferredFormation', ''),
                                  'country': manager_json.get('country', {}).get('name', ''),
                                  'date_birth': manager_json.get('dateOfBirthTimestamp', 0),
                                  'matches': manager_json.get('performance', {}).get('total', 0),
                                  'wins': manager_json.get('performance', {}).get('wins', 0),
                                  'draws': manager_json.get('performance', {}).get('draws', 0),
                                  'losses': manager_json.get('performance', {}).get('losses', 0),
                                  'goals_for': manager_json.get('performance', {}).get('goalsScored', 0),
                                  'goals_against': manager_json.get('performance', {}).get('goalsConceded', 0),
                                  'points': manager_json.get('performance', {}).get('totalPoints', 0)})
            
    return pd.DataFrame(list_managers)

# Obtención de las alineaciones y estadísticas de los jugadores
def get_single_team_lineups(lineups_dict: dict, home_away: bool, home_team: str, away_team: str) -> Tuple[str, pd.DataFrame]:

    # Formación a devolver
    formation = lineups_dict.get('formation', '')

    lineup = []
    for player in lineups_dict.get('players', []):
        
        # Diccionario con información del jugador
        player_dict = {'id': player.get('player', {}).get('id', 0), 
                       'team': home_team if home_away else away_team,
                       'opponent': away_team if home_away else home_team,
                       'player': player.get('player', {}).get('name', ''),
                       'position': player.get('position', ''),
                       'shirt_number': player.get('shirtNumber', 0)}
        
        # Para cada estadística, la concatenamos
        statistics = player.get('statistics', {})
        for st in statistics.keys():
            if st not in ['ratingVersions', 'statisticsType']:
                player_dict[st] = statistics[st]

        lineup.append(player_dict)
    
    # DF y quitamos minutos cero
    lineup_df = pd.DataFrame(lineup).fillna(0)
    lineup_df = lineup_df[lineup_df['minutesPlayed'] > 0]

    return formation, lineup_df

# Función para processar un simple partido
def get_matches_info(raw_matches_m_path: str) -> Tuple[pd.DataFrame, pd.DataFrame]: 

    # Listas para concatenar info
    match_info_dict = []
    lineups_list = []

    # Para cada partido, obtenemos los datos
    for match in os.listdir(raw_matches_m_path):
        
        # Full path i JSON
        full_path = os.path.join(raw_matches_m_path, match)
        match_json = json_to_dict(full_path)

        # Equipos y ID
        match_id = match_json.get('match', {}).get('event', {}).get('id', 0)
        home_team = match_json.get('match', {}).get('event', {}).get('homeTeam', {}).get('name', '')
        away_team = match_json.get('match', {}).get('event', {}).get('awayTeam', {}).get('name', '')

        # Información del partido
        match_dict = {'id': match_id,
                      'slug': match_json.get('match', {}).get('event', {}).get('slug', ''),
                      'round': match_json.get('match', {}).get('event', {}).get('roundInfo', {}).get('round', 0),
                      'venue': match_json.get('match', {}).get('event', {}).get('venue', {}).get('name', ''),
                      'attendance': match_json.get('match', {}).get('event', {}).get('attendance'),
                      'referee': match_json.get('match', {}).get('event', {}).get('referee', {}).get('name', ''),
                      'home_team': home_team,
                      'away_team': away_team,
                      'home_score': match_json.get('match', {}).get('event', {}).get('homeScore', {}).get('display', 0),
                      'away_score': match_json.get('match', {}).get('event', {}).get('awayScore', {}).get('display', 0)}

        # Alineaciones locales y vistantes
        home_formation, home_lineup = get_single_team_lineups(lineups_dict=match_json.get('lineups', {}).get('home', {}), home_away=1, home_team=home_team, away_team=away_team)
        away_formation, away_lineup = get_single_team_lineups(lineups_dict=match_json.get('lineups', {}).get('away', {}), home_away=0, home_team=home_team, away_team=away_team)

        # Añadimos formaciones al diccionario
        match_dict['home_formation'] = home_formation
        match_dict['away_formation'] = away_formation

        # Concatenamos informaciones
        match_info_dict.append(match_dict)
        lineups_list.append([home_lineup, away_lineup])
        
    # Dataframes
    match_info_df = pd.DataFrame(match_info_dict)
    # lineups_df = pd.concat(lineups_list, ignore_index=True)
    
    return match_info_df, lineups_list

In [101]:
league_id = 73
raw_out_path = r'C:\Users\xrosinach\Desktop\TFM-Scouting-FC-Barcelona\data\raw'
clean_out_path = r'C:\Users\xrosinach\Desktop\TFM-Scouting-FC-Barcelona\data\clean'

# Obtenemos el nombre de la liga y el path -> creación de la carpeta de output (clean)
league_name = comps[comps['id'] == league_id]['tournament'].iloc[0]
league_slug = league_name.lower().replace(' ', '-')
out_league_path = os.path.join(clean_out_path, 'ss', league_slug)
os.makedirs(out_league_path, exist_ok=True)

# Capeta de la liga y carpetas (seasons) dentro
raw_data_path = os.path.join(raw_out_path, 'ss', league_slug)
av_seasons_path = os.path.join(raw_data_path, 'available_seasons.json')
av_seasons = json_to_dict(json_path=av_seasons_path)

# Obtención del dataframe y guardado
av_seasons_df = get_seasons_df(league_id=league_id, available_seasons_json=av_seasons)
av_seasons_df.to_csv(os.path.join(out_league_path, 'available_seasons.csv'), sep=';', index=False)

# Para cada temporada
for season in av_seasons_df['year'].tolist():

    if not os.path.exists(os.path.join(raw_data_path, season)):
        continue

    # Carpeta de output de la temporada y otras carpetas que necesitaremos
    out_season_path = os.path.join(out_league_path, season)
    os.makedirs(out_season_path, exist_ok=True)

    # Info
    info_path = os.path.join(out_season_path, 'info')
    os.makedirs(info_path, exist_ok=True)

    # Standings
    standings_path = os.path.join(info_path, 'standings')
    os.makedirs(standings_path, exist_ok=True)

    # Match
    match_path = os.path.join(out_season_path, 'match')
    os.makedirs(match_path, exist_ok=True)

    # Matches stats
    matches_stats_path = os.path.join(match_path, 'stats')
    os.makedirs(matches_stats_path, exist_ok=True)

    # Carpeta con la información, lectura de los ficheros json
    raw_info_path = os.path.join(raw_data_path, season, 'info')
    standings_json = json_to_dict(json_path=os.path.join(raw_info_path, 'standings.json'))
    player_json = json_to_dict(json_path=os.path.join(raw_info_path, 'player.json'))
    team_json = json_to_dict(json_path=os.path.join(raw_info_path, 'team.json'))
    venue_json = json_to_dict(json_path=os.path.join(raw_info_path, 'venue.json'))

    # Tablas de clasificación y guardado
    total_st_df, home_st_df, away_st_df = obtain_standings_tables(league_id=league_id, season_key=season, standings_json=standings_json)
    total_st_df.to_csv(os.path.join(standings_path, 'total.csv'), sep=';', index=False)
    home_st_df.to_csv(os.path.join(standings_path, 'home.csv'), sep=';', index=False)
    away_st_df.to_csv(os.path.join(standings_path, 'away.csv'), sep=';', index=False)

    # Procesado de jugadores
    players_df = players_processing(player_json=player_json, players_dict_path=os.path.join(raw_info_path, 'player'))
    players_df.insert(0, 'league', league_id)
    players_df.insert(1, 'season', season)
    players_df.to_csv(os.path.join(info_path, 'player.csv'), sep=';', index=False)

    # Procesado de equipos
    teams_df = teams_processing(team_json=team_json, teams_dict_path=os.path.join(raw_info_path, 'team'))
    teams_df.insert(0, 'league', league_id)
    teams_df.insert(1, 'season', season)
    teams_df.to_csv(os.path.join(info_path, 'team.csv'), sep=';', index=False)

    # Procesado de estadios
    venues_df = venues_processing(venue_json=venue_json, venues_dict_path=os.path.join(raw_info_path, 'venue'))
    venues_df.insert(0, 'league', league_id)
    venues_df.insert(1, 'season', season)
    venues_df.to_csv(os.path.join(info_path, 'venue.csv'), sep=';', index=False)

    # Procesado de entrenadores
    managers_df = managers_processing(managers_dict_path=os.path.join(raw_info_path, 'manager'))
    managers_df.insert(0, 'league', league_id)
    managers_df.insert(1, 'season', season)
    managers_df.to_csv(os.path.join(info_path, 'manager.csv'), sep=';', index=False)

    # Paths con información
    raw_matches_path = os.path.join(raw_data_path, season, 'matches')
    raw_matches_m_path = os.path.join(raw_matches_path, 'match')
    raw_matches_p_path = os.path.join(raw_matches_path, 'player')

    # Información de los partidos
    match_info_df, lineups_df = get_matches_info(raw_matches_m_path=raw_matches_m_path)
    # Guardar

In [102]:
lineups_df

[[         id       team  opponent              player position  shirt_number  \
  0    930267  Barcelona  Mallorca         Joan García        G            13   
  1    827212  Barcelona  Mallorca        Jules Koundé        D            23   
  2   1402913  Barcelona  Mallorca         Pau Cubarsí        D             5   
  3    876214  Barcelona  Mallorca         Eric García        D            24   
  4    997035  Barcelona  Mallorca     Alejandro Balde        D             3   
  5   1153270  Barcelona  Mallorca        Fermín López        M            16   
  6   1000483  Barcelona  Mallorca         Marc Casadó        M            17   
  7    789071  Barcelona  Mallorca           Dani Olmo        M            20   
  8   1402912  Barcelona  Mallorca        Lamine Yamal        F            10   
  9     41789  Barcelona  Mallorca  Robert Lewandowski        F             9   
  10   814590  Barcelona  Mallorca     Marcus Rashford        F            14   
  11  1526618  Barcelona  Ma

In [ ]:
season = '2526'

# Paths con información
raw_matches_path = os.path.join(raw_data_path, season, 'matches')
raw_matches_m_path = os.path.join(raw_matches_path, 'match')
raw_matches_p_path = os.path.join(raw_matches_path, 'player')

In [ ]:
match_info_dict = []

# Para cada partido, obtenemos los datos
for match in os.listdir(raw_matches_m_path):
    
    # Full path i JSON
    full_path = os.path.join(raw_matches_m_path, match)
    match_json = json_to_dict(full_path)

    # Información del partido
    match_info_dict.append({'id': match_json.get('match', {}).get('event', {}).get('id', 0),
                            'slug': match_json.get('match', {}).get('event', {}).get('slug', ''),
                            'round': match_json.get('match', {}).get('event', {}).get('roundInfo', {}).get('round', 0),
                            'venue': match_json.get('match', {}).get('event', {}).get('venue', {}).get('name', ''),
                            'attendance': match_json.get('match', {}).get('event', {}).get('attendance'),
                            'referee': match_json.get('match', {}).get('event', {}).get('referee', {}).get('name', ''),
                            'home_team': match_json.get('match', {}).get('event', {}).get('homeTeam', {}).get('name', ''),
                            'away_team': match_json.get('match', {}).get('event', {}).get('awayTeam', {}).get('name', ''),
                            'home_score': match_json.get('match', {}).get('event', {}).get('homeScore', {}).get('display', 0),
                            'away_score': match_json.get('match', {}).get('event', {}).get('awayScore', {}).get('display', 0)})
    
# return pd.DataFrame(match_info_dict)

In [ ]:
get_single_team_lineups(match_json.get('lineups').get('home'), 1, 'a', 'b')

,id,team,opponent,player,position,shirt_number,totalPass,accuratePass,totalLongBalls,accurateLongBalls,...,expectedGoals,shotValueNormalized,totalOffside,bigChanceCreated,unsuccessfulTouch,dispossessed,onTargetScoringAttempt,goals,expectedGoalsOnTarget,bigChanceMissed
0,791773,a,b,Álex Remiro,G,1,40.0,24.0,24.0,8.0,...,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
1,353250,a,b,Álvaro Odriozola,D,20,12.0,10.0,3.0,3.0,...,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
2,838159,a,b,Igor Zubeldia,D,5,37.0,31.0,9.0,3.0,...,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
3,257091,a,b,Duje Ćaleta-Car,D,16,21.0,16.0,1.0,0.0,...,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
4,855835,a,b,Sergio Gómez,D,17,26.0,21.0,2.0,0.0,...,0.0946,-0.51,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
5,280979,a,b,Gonçalo Guedes,M,11,16.0,11.0,1.0,1.0,...,0.0000,0.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0
6,1216309,a,b,Jon Gorrotxategi,M,4,28.0,21.0,1.0,0.0,...,0.0298,-0.15,0.0,1.0,1.0,0.0,0.0,0.0,0.0000,0.0
7,832427,a,b,Carlos Soler,M,18,20.0,15.0,4.0,1.0,...,0.0000,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0000,0.0
8,1139409,a,b,Pablo Marín,M,15,11.0,9.0,0.0,0.0,...,0.0000,0.00,1.0,0.0,5.0,1.0,0.0,0.0,0.0000,0.0
9,949156,a,b,Luka Sučić,F,24,6.0,5.0,1.0,1.0,...,0.3020,0.51,0.0,0.0,0.0,0.0,1.0,1.0,0.8823,0.0


,id,slug,round,venue,attendance,referee,home_team,away_team,home_score,away_score
0,14083269,mallorca-barcelona,23,Spotify Camp Nou,44301,Alejandro Quintero Gonzalez,Barcelona,Mallorca,3,0
1,14083292,elche-real-sociedad,23,Reale Arena,30187,Jose Luis Guzman Mansilla,Real Sociedad,Elche,3,1


In [ ]:
match_info_dict

[{'id': 14083269,
  'slug': 'mallorca-barcelona',
  'round': 23,
  'venue': 'Spotify Camp Nou',
  'attendance': 44301,
  'referee': 'Alejandro Quintero Gonzalez',
  'home_team': 'Barcelona',
  'away_team': 'Mallorca',
  'home_score': 3,
  'away_score': 0},
 {'id': 14083292,
  'slug': 'elche-real-sociedad',
  'round': 23,
  'venue': 'Reale Arena',
  'attendance': 30187,
  'referee': 'Jose Luis Guzman Mansilla',
  'home_team': 'Real Sociedad',
  'away_team': 'Elche',
  'home_score': 3,
  'away_score': 1}]

In [ ]:
match_json.get('match', {}).get('event', {}).get('startTimestamp')

1770494400

In [ ]:
match_json.get('match', {}).get('event', {}).keys()

dict_keys(['eventState', 'correctAiInsight', 'correctHalftimeAiInsight', 'tournament', 'season', 'roundInfo', 'customId', 'status', 'winnerCode', 'attendance', 'venue', 'referee', 'homeTeam', 'awayTeam', 'homeScore', 'awayScore', 'time', 'changes', 'hasGlobalHighlights', 'hasXg', 'hasEventPlayerStatistics', 'hasEventPlayerHeatMap', 'detailId', 'crowdsourcingDataDisplayEnabled', 'id', 'defaultPeriodCount', 'defaultPeriodLength', 'defaultOvertimeLength', 'slug', 'currentPeriodStartTimestamp', 'startTimestamp', 'finalResultOnly', 'feedLocked', 'seasonStatisticsType', 'showTotoPromo', 'isEditor', 'eventFilters'])

In [ ]:
match_json.keys()

dict_keys(['match', 'lineups', 'statistics', 'shotmap', 'graph', 'incidents'])